In [2]:
import requests
import psycopg2
import pandas as pd
import json

In [10]:
db_params = {
    'host': 'localhost',
    'database': 'gps_onibus_rj',
    'user': 'postgres',
    'password': 'admin'
}

def fetch_data(datahora_group, type):
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    
    # Query to fetch data for the given datahora_group
    if type == 'lat_long':
        query = f"SELECT id, latitude_predicted, longitude_predicted FROM predictions_made WHERE datahora_group = '{datahora_group}'"
    else:
        query = f"SELECT id, datahora_predicted FROM predictions_made WHERE datahora_group = '{datahora_group}'"
    cursor.execute(query)
    rows = cursor.fetchall()
    
    conn.close()
    return rows

In [4]:
def make_post_request(datahora_group, previsoes):
    payload = {
        "aluno": "Victor Ribeiro Alves",
        "datahora": datahora_group,
        "previsoes": previsoes,
        "senha": ""
    }
    
    headers = {'Content-Type': 'application/json'}
    response = requests.post("", json=payload, headers=headers)
    
    if response.status_code == 200:
        print(f"POST request successful for {datahora_group}!")
        return response.text
    else:
        print(f"POST request failed for {datahora_group} with status code: {response.status_code}")
        return response.text

In [5]:
def save_lat_long_predictions():
    datahora_group_list = ['2024-05-16_08', '2024-05-16_16',
                        '2024-05-17_07', '2024-05-17_15',
                        '2024-05-18_12', '2024-05-18_20',
                        '2024-05-19_11', '2024-05-19_19',
                        '2024-05-20_09', '2024-05-20_17']
    responses_list = []
    for datahora_group in datahora_group_list:
        previsoes = fetch_data(datahora_group, 'lat_long')
        avaliacao_response = make_post_request(datahora_group.replace('_', ' '), previsoes)
        responses_list.append(avaliacao_response)
    data_dicts = [json.loads(json_str) for json_str in responses_list]
    pd.DataFrame(data_dicts).to_csv('results_predict_lat_long.csv')

In [13]:
def save_datahora_predictions():
    datahora_group_list = ['2024-05-16_12', '2024-05-16_20',
                        '2024-05-17_11', '2024-05-17_19',
                        '2024-05-18_08', '2024-05-18_16',
                        '2024-05-19_07', '2024-05-19_15',
                        '2024-05-20_13', '2024-05-20_21']
    responses_list = []
    for datahora_group in datahora_group_list:
        previsoes = fetch_data(datahora_group, 'datahora')
        avaliacao_response = make_post_request(datahora_group.replace('_', ' '), previsoes)
        responses_list.append(avaliacao_response)
    data_dicts = [json.loads(json_str) for json_str in responses_list]
    pd.DataFrame(data_dicts).to_csv('results_predict_datahora.csv')

In [14]:
save_datahora_predictions()

POST request successful for 2024-05-16 12!
POST request successful for 2024-05-16 20!
POST request successful for 2024-05-17 11!
POST request successful for 2024-05-17 19!
POST request successful for 2024-05-18 08!
POST request successful for 2024-05-18 16!
POST request successful for 2024-05-19 07!
POST request successful for 2024-05-19 15!
POST request successful for 2024-05-20 13!
POST request successful for 2024-05-20 21!
